In [1]:
import json
from os.path import join
from collections import Counter

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


In [23]:
def extract_triplets(D,A):
    questions, image_ids, answers, mcs =[],[],[],[]
    for d,a in zip(D['questions'],A['annotations']):
        assert d['image_id'] == a['image_id']
        questions.append(d['question'])
        image_ids.append(d['image_id'])
        answers.append(a['multiple_choice_answer'])
        mcs.append(' | '.join(d['multiple_choices']))
    print("%d questions (%d different)" % (len(questions), len(set(questions))))
    print("%d images (%d different)" % (len(image_ids), len(set(image_ids))))
    return questions, image_ids, answers, mcs

def extract_spatial_triplets(questions,image_ids,answers,mcs):
    words = open('spatial_words.txt','r').read().split()

    IQA = set()
    for q,i,a,mc in zip(questions, image_ids,answers,mcs):
        qwords = set(q.lower().strip('?').split())
        if len(qwords.intersection(words))>=1:
            IQA.add((i,q,a,mc))
    spatial_questions = []
    spatial_image_ids = []
    spatial_answers = []
    spatial_mcs = []
    for i,q,a,mc in IQA:
        spatial_questions.append(q)
        spatial_image_ids.append(i)
        spatial_answers.append(a)
        spatial_mcs.append(mc)
    print("%d questions (%d different)" % (len(spatial_questions), len(set(spatial_questions))))
    print("%d images (%d different)" % (len(spatial_image_ids), len(set(spatial_image_ids))))
    return spatial_questions, spatial_image_ids, spatial_answers, spatial_mcs

def write_triplets(spatial_questions,spatial_image_ids,spatial_answers,spatial_mcs,split):
    image_file = open('datasets/vqa/'+split+'/img_ids.txt','w',encoding='utf-8')
    question_file = open('datasets/vqa/'+split+'/questions.txt','w',encoding='utf-8')
    answer_file = open('datasets/vqa/'+split+'/answers.txt','w',encoding='utf-8')
    mc_file = open('datasets/vqa/'+split+'/mcs.txt','w',encoding='utf-8')
    for i,q,a,mc in zip(spatial_image_ids, spatial_questions, spatial_answers,spatial_mcs):
        image_file.write(str(i) + '\n')
        question_file.write(q.lower().strip('?') + '\n')
        mc_file.write(mc+'\n')
        answer_file.write(a +'\n')
    image_file.close()
    question_file.close()
    answer_file.close()
    mc_file.close()
    return 'done'

In [24]:
for split in 'train val'.split():
    D = json.loads(open('datasets/vqa/'+split+'/MultipleChoice_mscoco_'+split+'2014_questions.json').read())
    A = json.loads(open('datasets/vqa/'+split+'/mscoco_'+split+'2014_annotations.json').read())
    O = json.loads(open('datasets/vqa/'+split+'/OpenEnded_mscoco_'+split+'2014_questions.json').read())
    questions,image_ids,answers,mcs = extract_triplets(D,A)
    write_triplets(questions,image_ids,answers,mcs,split)

248349 questions (152050 different)
248349 images (82783 different)
121512 questions (81565 different)
121512 images (40504 different)
